In [1]:
import os
import time
import threading

In [4]:
print('Program enter')
print(f'Process id: {os.getpid()}')
print(f'Current thread name: {threading.current_thread().name}')
print('Sleeping')
time.sleep(5)
threading.current_thread().name = 'MyThread'
print(f'Current thread name: {threading.current_thread().name}')
print('Program exit')

Program enter
Process id: 2206
Current thread name: MainThread
Sleeping
Current thread name: MyThread
Program exit


#### Create a Thread
- Create an instance of Thread class
- Write a sub-class of Thread and instantiate the sub-class

In [22]:
def run():
    for i in range(1,6):
        print(f'{i} --> {threading.current_thread().name} -- Process id: {os.getpid()}')
        time.sleep(1)

In [29]:
print(f'Main Thread enter - {os.getpid()}')

t1 = threading.Thread(target=run, name='Thread-01')
t2 = threading.Thread(target=run, name='Thread-02')

t1.start()
t2.start()

t1.join() # Current thread (Main thread) goes to wait state, gets notified when t1 is dead
t2.join()

print('Main Thread exit')

Main Thread enter - 2206
1 --> Thread-01 -- Process id: 2206
1 --> Thread-02 -- Process id: 2206
2 --> Thread-01 -- Process id: 2206
2 --> Thread-02 -- Process id: 2206
3 --> Thread-01 -- Process id: 2206
3 --> Thread-02 -- Process id: 2206
4 --> Thread-01 -- Process id: 2206
4 --> Thread-02 -- Process id: 2206
5 --> Thread-01 -- Process id: 2206
5 --> Thread-02 -- Process id: 2206
Main Thread exit


#### Create a Thread whose target function accepts numbers, adds them and display the sum 

In [30]:
def add(*args):
    print(f'Thread name: {threading.current_thread().name}, Sum: {sum(args)}')

In [33]:
t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t2 = threading.Thread(target=add, name='Second', args=(11,22,33))

t1.start()
t2.start()

Thread name: First, Sum: 15
Thread name: Second, Sum: 66


#### Create a Thread by Inheritance
- Write a sub-class Thread 
- Target method by default is the instance method named 'run', we override this method
- In case we wish to have some other target method, then set the target using t.run attribute
- Arguments can be set explicitly using t.args attribute and access through self within the class (self.args)

In [45]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} --> {threading.current_thread().name} -- Process id: {os.getpid()}')
            time.sleep(1)

In [47]:
t1 = MyThread()
t1.name = 'MyThread-01'
t1.run = t1.do_something # setting the target
t1.start()

1 --> MyThread-01 -- Process id: 2206
2 --> MyThread-01 -- Process id: 2206
3 --> MyThread-01 -- Process id: 2206
4 --> MyThread-01 -- Process id: 2206
5 --> MyThread-01 -- Process id: 2206


#### Create a sub-class of Thread, name it as AddThread
- the target method would sum the arguments and display the sum

In [49]:
class AddThread(threading.Thread):
    def run(self):
        print(f'{threading.current_thread().name} --> Sum: {sum(self.args)}')

t1 = AddThread()
t1.name = 'MyThread-1'
t1.args = (1,2,3,4,5)

t2 = AddThread()
t2.name = 'MyThread-2'
t2.args = (11,22,33)

t1.start()
t2.start()

MyThread-1 --> Sum: 15
MyThread-2 --> Sum: 66


#### Thread assignment
1. Write a class named Resource (not a sub-class of Thread) 
2. It will have an instance field named 'data' (initialize it to 0)
3. It will have an instance method named 'do_something' 
4. The method will increment data by one, and display the name of the current thread followed by value of data
5. The method 'do_something' will be target for a Thread

In [83]:
class Resource:
    def __init__(self):
        self.data = 0
        
    def do_something(self):
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')

In [78]:
# Thread safe code
class Resource:
    def __init__(self):
        self.lock = threading.Lock()
        self.data = 0
        
    def do_something(self):
        self.lock.acquire()
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')
        self.lock.release()        

In [86]:
# Thread safe code
class Resource:
    def __init__(self):
        self.lock = threading.Lock()
        self.data = 0
        
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(1000000):
                pass
            print(f'{threading.current_thread().name} --> {self.data}')

In [84]:
r = Resource()

t1 = threading.Thread(target = r.do_something, name='First')
t2 = threading.Thread(target = r.do_something, name='Second')
t3 = threading.Thread(target = r.do_something, name='Third')

t1.start()
t2.start()
t3.start()

First --> 1
Second --> 2
Third --> 3


In [87]:
r = Resource()

for i in range(1,11):
    t = threading.Thread(target = r.do_something, name=f'Thread-{i}')
    t.start()

Thread-1 --> 1
Thread-2 --> 2
Thread-3 --> 3
Thread-4 --> 4
Thread-5 --> 5
Thread-6 --> 6
Thread-7 --> 7
Thread-8 --> 8
Thread-9 --> 9
Thread-10 --> 10
